基于Python字节码的虚拟机的设计与实现 
========

中期答辩 --- 陶青云
***

# Python的执行过程
***

# Python的字节码
***

# vm.py

In [29]:
import vm

# 实例化一个VirtualMachine对象
vm_obj = vm.VirtualMachine()

***
# 一个例子(斐波那契数列)

In [57]:
import dis
s = '''\
def fib(n):
    if n in (0,1):
        return n
    else:
        return fib(n-1) + fib(n-2)
        
for i in range(22):
    print fib(i),
'''
co = compile(s, "", "exec")
#dis.dis(co)

In [60]:
vm_obj.run_code(co)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946


In [63]:
exec co

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 6765 10946


***
# class VirtualMachine
完成虚拟机的主要功能。具体包括：
- 140多条字节码的具体实现
- 对函数调用栈的相关操作
- 维护一个指令指针IP
- 一个虚拟的CPU，包括3个功能
  * 取下一条指令
  * 解码操作数
  * 执行指令
  

In [30]:
import re
# VirtualMachine类中所有以大写字母开头方法的都是相关字节码的实现
byte_codes = [ k for k in vm.VirtualMachine.__dict__.keys() if re.match('[A-Z]', k)]
#print "\n".join(byte_codes)


In [31]:
# 用一个列表结构来维护函数调用栈帧
#print vm_obj.frames

# 相关操作有
#print vm_obj.make_frame, vm_obj.pop_frame, vm_obj.push_frame, vm_obj.run_frame

# 指令指针在每个frame中维护

In [32]:
vm_obj.run_code
vm_obj.run_frame
vm_obj.parse_byte
vm_obj.dispatch

<bound method VirtualMachine.dispatch of <vm.VirtualMachine object at 0x00000000043F8B00>>

***
# class Frame
Frame对象对应函数调用时所动态生成的栈帧（与C语言栈帧类似）。frame是一个属性的集合，它没有任何方法。
- 它要维护3重要的命名空间，全局，局部和内建命名空间。
- 它还要包含要执行的指令，和指令指针。
- 数据栈
- 块栈（ 与循环结构，异常处理相关）
- 上一个frame的引用  

与C语言不同，函数执行返回后，Frame可能不会被销毁。这点是实现Generator的基础

```python
class Frame(object):
    def __init__(self, f_code, f_globals, f_locals, f_back):
        self.f_code = f_code        # 指令集
        self.f_globals = f_globals    # 全局命名空间
        self.f_locals = f_locals     # 局部命名空间
        self.f_back = f_back        # 上一个frame的引用
        if hasattr(__builtins__, '__dict__'):   #内建命令空间
            self.f_builtins = __builtins__.__dict__ 
        else:
            self.f_builtins = __builtins__
        self.stack = []            # 数据栈       
        self.f_lasti = 0           # 指令指针
        self.block_stack = []        # 块栈
        
        ##---------------------------- generator，closure相关
        self.generator = None
        if f_code.co_cellvars:
            self.cells = {}
            for var in f_code.co_cellvars:
                cell = self.f_locals.get(var)
                self.cells[var] = cell
        else:
            self.cells = None
            
        if f_code.co_freevars:
            if not self.cells:
                self.cells = {}
            for var in f_code.co_freevars:
                self.cells[var] = self.f_locals.get(var)

 ``` 

In [33]:
print vm_obj.frame == None

True


***
# class Function
 Fuction对象代表Python中的一个函数。他的重点在于可调用性。
 当它被调用时，自动创建一个Frame对象执行函数中的字节码

In [28]:
vm.Function??

***
# 参数传递

In [73]:
def test(x, y=5, *args, **kwargs):
    #print x, y, args, kwargs
    pass

test(1)
test(1,2) 
test(1,2,3) 
test(1,2,3,4)
test(x=1) 
test(x=1,y=1) 
test(x=1,y=1,a=1)
test(x=1,y=1,a=1,b=1) 
test(1,y=1)
test(1,2,3,4,a=1) 
test(1,2,3,4,k=1,t=2,o=3)

与之相关的字节码
- CALL_FUNCTION(argc)
- CALL_FUNCTION_VAR(argc)
- CALL_FUNCTION_KW(argc)
- CALL_FUNCTION_VAR_KW(argc)

官方文档对[CALL_FUNCTION](https://docs.python.org/2/library/dis.html#opcode-CALL_FUNCTION)的解释
>    Calls a function. The low byte of argc indicates the number of positional parameters, the high byte the number of keyword parameters. On the stack, the opcode finds the keyword parameters first. For each keyword argument, the value is on top of the key. Below the keyword parameters, the positional parameters are on the stack, with the right-most parameter on top. Below the parameters, the function object to call is on the stack. Pops all function arguments, and the function itself off the stack, and pushes the return value.

In [71]:
vm_obj.CALL_FUNCTION??
vm_obj.call_function??

***
# 闭包机制的实现
>    Closures are functions that refer to independent (free) variables. In other words, the function defined in the closure 'remembers' the environment in which it was created. 
[MDN中的解释](https://developer.mozilla.org/en/docs/Web/JavaScript/Closures)

相关代码或字节码：
- Function.\_\_call\_\_
- LOAD_CLOSURE
- MAKE_CLOSURE


In [92]:
# 一个简单闭包的例子
def makeAdd(x):
    def f(y):
        return x + y
    return f

add1 = makeAdd(1)
add3 = makeAdd(3)
print add1(0)
print add3(0)

1
3


In [87]:
# Function在创建时会保存它的自由变量，在func_closure属性中
print add1.func_closure[0].cell_contents
print add3.func_closure[0].cell_contents

1
3


In [91]:
# 字节码层的解释
# 编译器在编译时已确定自由变量
print add1.__code__.co_freevars

# 同时外层函数也会确定哪些变量会被内函数引用
print makeAdd.__code__.co_cellvars

('x',)
('x',)


有了这些信息，就可以完成闭包机制。闭包函数在被调用时，在创建Frame之前，把自由变量和对应的值存入局部命名空间中。（CPython不是这样实现的，看不懂）

***
# 生成器

***
# 异常处理机制